# Git для Data Science: контроль версий данных с помощью Python и DVC

Данная публикация представляет собой частичный перевод Кристиана Иванчича [Data Version Control With Python and DVC](https://realpython.com/python-data-version-control/).

Машинное обучение и наука о данных сопряжены с рядом задач, отличающихся от традиционной разработки программного обеспечения. [Системы управления версиями](https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%8F%D0%BC%D0%B8) помогают разработчикам контролировать изменения исходного кода. Но управление версиями данных, изменения в моделях и датасетах, отлажено не так хорошо.

Если датасет изменился, точно воспроизвести эксперименты, проведенные вами или другими людьми, становится сложной задачей. Многие команды активно разрабатывают инструменты и фреймворки для решения этих проблем.

В этом руководстве мы узнаем как:
- использовать инструмент под названием DVC для решения некоторых из этих проблем;
- корректно отслеживать и редактировать наборы данных и модели;
- совместно использовать одну систему с товарищами по команде;
- делать воспроизводимые эксперименты по машинному обучению.

# Что такое система управления версиями данных

В стандартной разработке программного обеспечения многим людям необходимо работать с общей кодовой базой и обрабатывать несколько версий одного и того же кода. Это может быстро привести к путанице и дорогостоящим ошибкам. Чтобы решить эту проблему, разработчики используют системы управления версиями, такую как Git.

В системе управления версиями есть центральный репозиторий кода, который представляет текущее официальное состояние проекта. Разработчик может сделать копию этого проекта, внести в них изменения и запросить, чтобы новая версия кода стала официальной. В результате запроса код проверяется и тестируется, пока не будет выпущен.

Такие циклы могут происходить по многу раз в день в традиционных проектах, связанных с разработкой. Но в мире промышленного Data Science подобные соглашения и стандарты пока не закрепились. Наличие систем, которые позволяют людям работать быстро и продолжать с того места, на котором остановились другие, повысило бы скорость и качество получаемых результатов. Это позволит людям прозрачно управлять данными, эффективно проводить эксперименты и сотрудничать с другими людьми и командами.

Один из инструментов, помогающих исследователям управлять наборами данных и моделями и проводить воспроизводимые эксперименты — **DVC** (сокр. от Data Version Control).

# Что такое DVC

DVC – это инструмент командной строки, написанный на Python. Он имитирует команды и рабочие процессы Git, чтобы пользователи знакомые с Git, могли легче пользоваться этим инструментом. DVC предназначен для работы вместе с Git. Фактически, команды git и dvc часто используются вместе, одна за другой. В то время как Git используется для хранения версий кода, DVC делает то же самое для файлов моделей и датасетов.

Git может хранить код как локально, так и на хостинге GitHub, Bitbucket или GitLab. Аналогичным образом, DVC позволяет загружать данные на удаленный репозиторий для хранения данных и моделей. Вы можете создать локальную копию удаленного репозитория, изменить файлы, а затем загрузить свои изменения, чтобы поделиться ими с командой.

Удаленный репозиторий может находиться на том же компьютере, на котором вы работаете, или в облаке. DVC поддерживает большинство главных облачных провайдеров, включая AWS, GCP и Azure. Но вы можете настроить удаленный репозиторий DVC на любом другом сервере. С соблюдением мер предосторожности, которые не позволят другим участникам повредить или удалить данные.

При созранении данных в удаленном репозитории, создается `.dvc`-файл. Файл `.dvc` - это небольшой текстовый файл, который описывает текущее состояние файлов данных в удаленном хранилище. За счет небольшого размер `.dvc`-файл может храниться вместе с вашим программным кодом, например, на GitHub.

Далее мы на практике изучим наиболее важные функции, проработав несколько примеров. Перед тем как начать, нам нужно настроить среду для работы, а затем получить некоторые данные.

# Настраиваем рабочую среду

В этом руководстве мы рассмотрим, как использовать DVC, попрактиковавшись на примерах работы с данными изображений. Мы поиграем с большим количеством файлов изображений и обучим модель машинного обучения, которая распознает, что содержит изображение.

Для работы с примерами в системе должны быть установлены [Python 3](https://realpython.com/installing-python/) и [Git](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git).

Поскольку DVC – это инструмент командной строки, необходимо уметь работать в командной строке вашей операционной системы. Если вы пользователь Windows, ознакомьтесь с разделом [«Запуск DVC в Windows» (англ.)](https://dvc.org/doc/user-guide/running-dvc-on-windows).

Чтобы подготовить рабочую среду, нам нужно сделать следующие шаги:
1. Создать и активировать виртуальное окружение.
2. Установить DVC и необходимые библиотеки Python.
3. Создайть форк и клонировать GitHub-репозиторий со всем кодом.
4. Загрузить набор данных для использования в примерах.

Вы можете использовать любой менеджер пакетов и окружений, какой захотите. В этом руководстве мы используем `conda`, поскольку он отлично поддерживает инструменты для анализа данных и машинного обучения. Чтобы создать и активировать виртуальную среду, откройте интерфейс командной строки и введите следующую команду:

```bash
conda create --name dvc python=3.8.2 -y
```

Команда `create` создает новую виртуальную среду. Флаг `--name` дает имя среде – в данном случае `dvc`. Аргумент `python` позволяет выбрать версию Python, которую мы хотим установить в среде. Наконец, флаг `-y` автоматически соглашается установить все необходимые пакеты, которые нужны Python, без необходимости отвечать на какие-либо запросы.

Как только все установлено, активируем среду:

```bash
conda activate dvc
```

Теперь у нас есть среда Python, независимая от установленных в операционной системе библиотек Python. В нее мы установим внешние библиотеки:

- `dvc` – главный герой этого руководства;
- `scikit-learn` – библиотека machine learning для обучения моделей;
- `scikit-image` – библиотека обработки изображений, которую мы будем использовать для подготовки данных;
- `pandas` – библиотека анализа данных, которая организует данные в виде таблиц;
- `numpy` – библиотека, добавляющая поддержку многомерных данных, таких как изображения.

Их тоже можно установить с помощью `conda`:

```bash
conda config --add channels conda-forge
conda install dvc scikit-learn scikit-image pandas numpy
```

В качестве альтернативы можно использовать установщик pip:

```bash
python -m pip install dvc scikit-learn scikit-image pandas numpy
```

Теперь у нас есть все необходимые библиотеки Python для запуска кода. Теперь нам нужно сделать форк репозитория в свою учетную запись на GitHub. На странице репозитория GitHub нажмите `Fork` в правом верхнем углу экрана и выберите во всплывающем окне вашу учетную запись. GitHub создаст копию репозитория в вашей учетной записи.

Клонируем форк на свой компьютер с помощью команды `git clone` и переходим в папку репозитория (не забудьте поменять `YourUsername` на имя вашего аккаунта на GitHub):

```bash
git clone https://github.com/YourUsername/data-version-control
cd data-version-control
```

Cтруктура каталогов репозитория выглядит следующим образом:

```
data-version-control/
|
├── data/
│   ├── prepared/
│   └── raw/
|
├── metrics/
├── model/
└── src/
    ├── evaluate.py
    ├── prepare.py
    └── train.py

```

В репозитории шесть каталогов:

1. `src/` предназначен для исходного кода;
2. `data/` предназначен для всех версий набора данных;
3. `data/raw/` предназначен для данных, полученных из внешнего источника;
4. `data/prepare/` предназначен для данных, измененных внутри;
5. `model/` предназначен для моделей машинного обучения;
6. `data/metrics/` предназначен для отслеживания показателей производительности моделей.

Каталог `src/` уже содержит три файла Python:
1. `prepare.py` содержит код подготовки данных для обучения.
2. `train.py` содержит код обучения модели машинного обучения.
3. `evalueate.py` содержит код оценки результатов модели машинного обучения.

# Набор данных для обучения

Последний шаг в подготовке – получить пример набора данных, который мы можем использовать для практики DVC. Пример с изображениями подходит для этого лучше всего: управление большим количеством крупных файлов – это то, в чем отличается DVC. Мы будем использовать набор данных [Imagenette](https://github.com/fastai/imagenette), предоставляемый компанией [fastai](https://www.fast.ai/).

Imagenette – это подмножество набора данных [ImageNet](http://www.image-net.org/), который часто используется в качестве эталонного набора данных во многих статьях по машинному обучению. ImageNet слишком велик, чтобы использовать его в качестве примера, поэтому мы возьмем его подмножество Imagenette. Перейдите на [GitHub-страницу Imagenette](https://github.com/fastai/imagenette) и нажмите ссылку для загрузки `160 px download`.

Это загрузит набор данных, сжатый в архив TAR. Размер файла – порядка 100 Мб. Пользователи Mac могут извлечь файлы, дважды щелкнув архив в Finder. Пользователи Linux могут распаковать его с помощью команды `tar`. Пользователям Windows потребуется установить инструмент для распаковки файлов TAR, например [7-zip](https://www.7-zip.org/).

Набор данных структурирован определенным образом. В нем есть две основные папки:

1. `train/` включает изображения для обучения модели.
2. `val/` содержит изображения для валидации модели.

---
**Примечание**. Валидация обычно происходит *во время обучения* модели – так аналитики понимают, насколько хорошо модель обучается. Поскольку это руководство не ориентировано на показатели производительности, мы будете использовать набор для проверки модели *после ее обучения*.

---

Imagenette – это набор обучающих данных для задачи классификации: каждое изображение имеет связанный с ним класс, описывающий, что на нем изображено. Чтобы решить задачу классификации, необходимо обучить модель, которая может точно определять класс изображения.

Каталоги `train/` и `val/` содержат несколько папок. Каждая из них имеет код, который соответствует одному из 10 классов:

1. Линь (вид лучеперых рыб)
2. Английский спрингер-спаниель (порода собак)
3. Кассетный проигрыватель
4. Цепная пила
5. Церковь
6. Валторна
7. Мусоровоз
8. Топливораздаточная колонка
9. Мячик для гольфа
10. Парашют

Для простоты и скорости в руководстве мы будем обучать модель, используя лишь два из десяти классов: мячик для гольфа и парашют. После обучения модель должна приняв изображение, сообщить является ли изображение мячом для гольфа или изображением парашюта. Такую задача, когда модель выбирает между двумя типами объектов, называют бинарной классификацией.

Скопируем папки `train/` и `val/` и поместив их в репозитории data-version-control в каталог `data/raw/`. Структура репозитория теперь будет выглядеть так:

```bash
data-version-control/
|
├── data/
│   ├── prepared/
│   └── raw/
│       ├── train/
│       │   ├── n01440764/
│       │   ├── n02102040/
│       │   ├── n02979186/
│       │   ├── n03000684/
│       │   ├── n03028079/
│       │   ├── n03394916/
│       │   ├── n03417042/
│       │   ├── n03425413/
│       │   ├── n03445777/
│       │   └── n03888257/
|       |
│       └── val/
│           ├── n01440764/
│           ├── n02102040/
│           ├── n02979186/
│           ├── n03000684/
│           ├── n03028079/
│           ├── n03394916/
│           ├── n03417042/
│           ├── n03425413/
│           ├── n03445777/
│           └── n03888257/
|
├── metrics/
├── model/
└── src/
    ├── evaluate.py
    ├── prepare.py
    └── train.py
```

Отлично! Мы завершили настройку и готовы экспериментировать с DVC.

# Базовый рабочий процесс DVC

Скачанного нами набора данных достаточно, чтобы начать практиковаться в DVC. В этом разделе мы увидим, как DVC в тандеме с Git позволяет управлять и кодом, и данными.

Для начала создадим и переключимся на ветку нашего первого эксперимента:

```bash
git checkout -b "first_experiment"
```

`git checkout` изменяет текущую ветку, а переключатель -b сообщает Git, что этой ветки не существует и ее следует создать.

Далее нужно инициализировать DVC. Предварительно нужно убедиться, что мы находимся в папке верхнего уровня репозитория. Далее запускаем команду `dvc init`:

```bash
dvc init
```

Эта команда приведет к созданию папки `.dvc`, в которой хранится информация о конфигурации, аналогично тому как каталог `.git` содержит данные Git. В принципе, вам никогда не нужно открывать эту папку, если только вы не хотите узнать, как работает dvc под капотом.

---

**Примечание**: DVC недавно начал сбор анонимной аналитики использования, чтобы авторы могли лучше понять, как используется DVC. Это помогает им улучшить инструмент. Вы можете отключить эту функцию, установив для параметра конфигурации аналитики значение `false`:

```bash
dvc config core.analytics false
```

---

Git позволяет отправить локальный код в удаленный репозиторий так, чтобы у всех участников проекта был единый источник информации. Коллеги могут проверить ваш код и поработать над ним локально, не опасаясь испортить текст программы для всех остальных. То же справедливо и для DVC.

Теперь нам нужно удаленное хранилище файлов данных и моделей, контролируемых DVC. Пока что для обучения это может быть просто другая папка в вашей системе. Создадим каталог за пределами  репозитория `data-version-control/` и назовем его `dvc_remote`.

Вернемся в репозиторий `data-version-control/` и укажем DVC, где находится удаленное хранилище:

```bash
dvc remote add -d remote_storage path/to/your/dvc_remote
```

Теперь DVC знает, где сделать резервную копию данных и моделей. Команда `dvc remote add` сохраняет местоположение удаленного хранилища и называет его `remote_storage`. Если хотите, вы можете выбрать другое имя для переменной. Ключ `-d` сообщает DVC, что это удаленное хранилище по умолчанию (`default`). Можно добавить и другие удаленные хранилища и переключаться между ними.

Вы всегда можете проверить, какое удаленное хранлище используется репозиторием. Внутри папки `.dvc` находится файл `config`, в котором хранится информация о конфигурации репозитория:

```
[core]
    analytics = false
    remote = remote_storage
['remote "remote_storage"']
    url = /path/to/your/remote_storage
```

Как мы упоминали выше, DVC поддерживает множество облачных систем хранения, таких как  AWS S3, Google Cloud Storage и Microsoft Azure Blob Storage. Об их подключении можно узнать больше в [официальной документации DVC](https://dvc.org/doc/command-reference/remote/add).

Основное практическое правило, которому мы должны следовать, заключается в том, что небольшие файлы отправляются на GitHub, а большие файлы – в удаленное хранилище DVC.

# Отслеживание файлов

И Git, и DVC используют команду `add` для запуска отслеживания файлов. Добавим папки `train/` и `val/` в элемент управления DVC:

```
dvc add data/raw/train
dvc add data/raw/val
```

Вот что при этом делает DVC:

1. Добавляет каталоги `train/` и `val/` в `.gitignore`.
2. Создает два dvc-файла: `train.dvc` и `val.dvc`.
3. Копирует папки `train/` и `val/` в промежуточную область (staging).

`.gitignore` – это текстовый файл со списком файлов, которые Git не должен отслеживать. Когда файл указан в `.gitignore`, он невидим для команд `git`. Добавляя папки `train/` и `val/` в `.gitignore`, DVC гарантирует, что мы случайно не загрузим большие файлы данных на GitHub.

Вот так выглядил репозиторий до выполнения каких-либо команд

![](https://files.realpython.com/media/0_new-start.48d65eac9b3f.png)

Все, что контролирует DVC, находится слева (зеленый цвет), а все, что контролирует Git, – справа (синий). Упрощенно будем считать, что репозитории есть файл `code.py` с кодом Python и папка `train/` с данными для обучения.

Когда мы запускаем `dvc add train/`, папка с большими файлами переходит под контроль DVC, а маленькие dvc-файлы и файл `.gitignore` переходят под контроль Git. Папка `train/` попадает в промежуточную область (staging) DVC:

![](https://files.realpython.com/media/1_new-dvc_add.e7d290c59325.png)

После того, как большие файлы изображений будут помещены под управление DVC, можно добавить весь код и небольшие файлы под управления Git с помощью `git add`:


```bash
git add --all
```

Теперь все файлы находятся под управлением соответствующих систем контроля версий.

![](https://files.realpython.com/media/2_new-git_add.2de3d14865f7.png)

Если кто-то хочет работать над проектом и использовать данные `train/` и `val/`, им сначала нужно загрузить репозиторий Git, затем использовать файлы `.dvc` для получения текущей версии данных.

Но прежде чем люди смогут получить репозиторий и данные, необходимо загрузить файлы в удаленное хранилище.

# Загрузка файлов

Чтобы загрузить файлы на GitHub, нам нужно сначала создать «снимок» текущего состояния репозитория:

```bash
git commit -m "Начало работы с DVC: настройка и загрузка DVC-файлов"
```

Ключ `-m` означает, что последующий текст в кавычках представляет собой сообщение, объясняющее, что было сделано.

В DVC также есть команда `commit`, но она не делает то же самое, что `git commit`. DVC не нужен снимок всего репозитория. Инструмент позволяет загружать файлы, как только они отслеживаются с помощью `dvc add`.

Команда `dvc commit` используется при изменении уже отслеживаемого файла. Когда мы вносим локальное изменение в данные, мы должны зафиксировать это изменение в кеше перед загрузкой на удаленный компьютер. Так как мы пока не меняли данные с момента их добавления, `commit` для dvc делать не нужно.

---
Подробнее об отличиях `add` и `commit` в DVC и Git рассказывает [документаця DVC](https://dvc.org/doc/command-reference/add).

---

Чтобы загрузить файлы из кеша в удаленное хранилище, используем команду `push`:

```bash
dvc push
```

DVC просмотрит все папки локального репозитория в поисках dvc-файлов. Как уже упоминалось, эти файлы сообщают DVC, какие данные необходимо скопировать, и DVC копирует их из кеша в удаленное хранилище.

![](https://files.realpython.com/media/new-dvc_push_1.1e26af053dd1.png)

Теперь данные безопасно хранятся в удаленном репозитории. Осталось отправить файлы под управлением Git на GitHub:

```bash
git push --set-upstream origin first_experiment
```

GitHub не знает о новой ветке, которую мы создали локально, поэтому при первом `push` необходимо использовать параметр `--set-upstream`. Ваш код и другие небольшие файлы теперь безопасно хранятся на GitHub.

![](https://files.realpython.com/media/new-git_push_1.5ee03041c300.png)


Теперь и код, и данные доступны удаленно. 

# Скачивание файлов

Чтобы узнать, как загружать файлы, сначала нужно удалить некоторые из них из нашего репозитория.

Как только вы добавите свои данные с помощью `dvc add` и отправите их с помощью `dvc push`, они будут сохранены в резервной копии. Если вы хотите сэкономить место, вы можете удалить фактические данные. Пока все файлы отслеживаются DVC и dvc-файлы находятся в репозитории, мы можем быстро вернуть данные.

Например, мы можем удалить целиком папку `val/`:

```bash
rm -rf data/raw/val
```

Это приведет к удалению папки `data/raw/val/` из репозитория, но каталог по-прежнему безопасно хранится в кэше и удаленном хранилище. Чтобы вернуть данные из кеша, воспользуемся командой `dvc checkout`:

```bash
dvc checkout data/raw/val.dvc
```

Папка `data/raw/val/` будет восстановлена. Если вы хотите, чтобы DVC выполнил поиск по всему репозиторию и проверил, чего не хватает, достаточно запустить `dvc checkout` без дополнительных аргументов.

Когда вы клонируете свой репозиторий GitHub на новом компьютере, кеш будет пустым. Команда fetch загружает содержимое удаленного хранилища в кеш:

```bash
dvc fetch data/raw/val.dvc
```

Или вы можете использовать `dvc fetch`, чтобы получить данные для всех файлов DVC в репозитории. Как только данные окажутся в кеше, проверьте их в репозитории с помощью `dvc checkout`. Вы можете выполнить вместе `fetch` и `checkout` с помощью одной команды `dvc pull`. Эта команда копирует ваши данные с удаленного хранилища в кеш и репозиторий за один проход. Действия этих команд аналогично одноименным командам Git.


![](https://files.realpython.com/media/5_new-download.dd38540ba18f.png)

Имейте в виду, что сначала нужно получить dvc-файлы из Git, и только потом вы можете вызывать команды DVC. Если dvc-файлов нет в репозитории, программа DVC не будет знать, какие данные требуется получить.

Итак, мы рассмотрели базовый рабочий процесс взаимодействия DVC и Git. Всякий раз, когда мы добавляем дополнительные данные или изменяем код, необходимо запустить соответствующие команды `add`, `commit`, `push`, чтобы сохранить текущую версию. Многим людям этого базового рабочего процесса будет достаточно для повседневных нужд.

Остальная часть этого руководства посвящена конкретным случаям использования DVC для машинного обучения и Data Science.

# Построение модели машинного обучения

Используя набор данных Imagenette, мы научим модель различать изображения мячей для гольфа и парашютов. Нам нужно будет выполнить три основных шага:

1. Подготовить данные для обучения.
2. Обучить модель машинного обучения.
3. Оценить производительность модели.

Эти шаги соответствуют трем файлам Python в папке `src/`:

1. `prepare.py`
2. `train.py`
3. `evaluate.py`

В следующих подразделах объясняется, что делает каждый файл.

## Подготовка данных

Данные хранятся в нескольких каталогах. Чтобы упростить использование данных, мы создадим csv-файл, содержащий список изображений и их меток. CSV-файл будет содержать два столбца: столбец `filename`, содержащий полный путь для одного изображения, и столбец `label`, содержащий фактическую строку метки, например `"golf ball"` или `"parachute"`. Каждая строка соответствует одному изображению.

```csv
filename, label
full/path/to/data-version-control/raw/n03445777/n03445777_5768.JPEG,golf ball
full/path/to/data-version-control/raw/n03445777/n03445777_5768,golf ball
full/path/to/data-version-control/raw/n03445777/n03445777_11967.JPEG,golf ball
...
```